# Loadout

In [3]:
from abc import ABCMeta, abstractmethod, ABC
from pathlib import Path
from uuid import uuid4
from typing import Optional, Callable, List
import inspect
import asyncio

from ruamel.yaml import YAML

import panel as pn
from panel.viewable import Viewer

import param
from param import rx, Parameterized

from langchain_core.language_models import BaseLanguageModel
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma # Useless
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage

import chromadb
from chromadb.api.client import Client
from chromadb.api.models.Collection import Collection

import tiktoken

from dotenv import load_dotenv


###################Other Thangs#######################################
import sys
sys.path.append("..")

from pyllments.config import BASE_DIR

load_dotenv()
pn.extension()

In [4]:
%load_ext autoreload
%autoreload 2

# Interface Explore

In [16]:
spacer = pn.Spacer(
    height=100,
    styles={"background": "teal"},
)
spacer0 = pn.Spacer(
    height=100,
    styles={"background": "red"},
)

In [35]:
gspec = pn.GridSpec(width=800, height=600)

In [36]:
gspec[0, :] = spacer

In [39]:
gspec[1, :] = spacer0


    (1, 0): Spacer(height=300, styles={'background': 'teal'}, width=800)

The following shows a view of the grid (empty: 0, occupied: 1, overlapping: 2):

[[1]
 [2]]


In [45]:
pn.Column(spacer, pn.Spacer(height=20), spacer0)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e01e7589-7171-4850-87ae-6e5254501dfc': {'version…

In [15]:
clear_chat_button = pn.widgets.Button(name="Clear Chat", width=100)
model_chooser = pn.widgets.Select(name="LLM", options=["Model 1", "Model 2", "Model 3"])
corpus_chooser = pn.widgets.Select(
    name="Corpus", options=["Corpus 1", "Corpus 2", "Corpus 3"]
)

chat_sidebar = pn.Column(
    clear_chat_button, corpus_chooser, model_chooser, align="center"
)

In [16]:
chat_feed = pn.chat.ChatFeed(height=400, width=800)

chat_input = pn.chat.ChatAreaInput(width=600, rows=3)
send_button = pn.widgets.Button(name="Send", width=140, height=60)
toggle_retriever = pn.widgets.Toggle(name="Retriever", value=False, width=60, height=60)
chat_send_row = pn.Row(chat_input, send_button, toggle_retriever)

main_chat = pn.Column(chat_feed, chat_send_row)

In [17]:
chat_tab = pn.Row(sidebar, main_chat)
chat_tab

NameError: name 'sidebar' is not defined

In [106]:
new_corpus_button = pn.widgets.Button(name="New Corpus", width=100)
corpus_name_box = pn.widgets.TextInput(name="Corpus Name", width=200)

strategy_select = pn.widgets.Select(
    name="Strategy", options=["Strategy 1", "Strategy 2", "Strategy 3"]
)
parameters = pn.Column("## Parameters", styles={"background": "whitesmoke"})

corpus_left = pn.Column(new_corpus_button, corpus_name_box, strategy_select, parameters)
corpus_left

BokehModel(combine_events=True, render_bundle={'docs_json': {'e21f5372-801d-449c-b89a-d365f6e00af6': {'version…

In [103]:
current_corpus_name = pn.pane.Markdown(object="# Current Corpus: None")

file_input = pn.widgets.FileInput(name="Upload File")
file_input_button = pn.widgets.Button(name="Upload File", width=100)
file_input_row = pn.Row(file_input, file_input_button)

site_loader_input = pn.widgets.TextInput(
    name="Site Loader", value="https://www.example.com"
)
site_loader_button = pn.widgets.Button(name="Load Site", width=100)
site_loader_row = pn.Row(site_loader_input, site_loader_button)


corpus_mid = pn.Column(
    current_corpus_name, corpus_chooser, file_input_row, site_loader_row
)
corpus_mid

BokehModel(combine_events=True, render_bundle={'docs_json': {'681b277e-fb44-42a9-b162-b028bf324e3d': {'version…

In [105]:
resource_list_to_process = pn.Column(
    "# Resources to Process", styles={"background": "whitesmoke"}
)
resource_list_process = pn.widgets.Button(name="Process", width=100)

resource_list = pn.Column("# Resources Present", styles={"background": "whitesmoke"})

resource_progress = pn.pane.Markdown(object="## Progress")

corpus_right = pn.Column(
    resource_list_to_process, resource_list_process, resource_list, resource_progress
)
corpus_right

BokehModel(combine_events=True, render_bundle={'docs_json': {'6427310c-e1b1-4253-a325-6dbf07207814': {'version…

In [107]:
corpus_tab = pn.Row(corpus_left, corpus_mid, corpus_right)
corpus_tab

BokehModel(combine_events=True, render_bundle={'docs_json': {'5eb51c56-81c7-452b-bf7d-75dffd9d9e54': {'version…

In [116]:
srv.stop()

AssertionError: Already stopped

In [135]:
pn.config.theme = "default"

In [133]:
serve(
    pn.Tabs(
        ("# Chat", chat_tab),
        ("Corpus Settings", corpus_tab),
        stylesheets=[".bk-tab {font-size: 20px}"],
    )
)

Launching server at http://localhost:35594


In [42]:
srv = None


def serve(obj):
    global srv
    try:
        srv.stop()
    except:
        srv = pn.serve(obj, port=35594)
        return srv

# Business Logic Class Construction

## Chat model

In [ ]:
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")
chat_model

In [56]:
chat_model = ChatOpenAI()
# To get the arguments of the chat model, you can use the 'inspect' module
import inspect

print(inspect.getfullargspec(chat_model.__init__))

FullArgSpec(args=['__pydantic_self__'], varargs=None, varkw='data', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={'return': None, 'data': typing.Any})


In [37]:
class LLMChatModel(param.Parameterized):
    model_class = param.ClassSelector(
        class_=BaseLanguageModel,
        is_instance=False,
        doc='Class of a LangChain Chat Model')
    model_args = param.ClassSelector(
        class_=(dict, list),
        doc='''Takes a dictionary of arguments to pass to expose and send to the
        model class. If you set a None value as the key, the argument will be exposed,
        with the default value set. Passing a list is the same as passing a dict
        with all values set to None.''',
        is_instance=True)

    model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self._set_params()
        self._create_model()

    def _set_params(self):
        """Handles setting the params on init"""
        if self.model_args:
            for arg, val in self.model_args.items():
                if arg in self.model_class.__fields__:
                    if val is None:
                        default = self.model_class.__fields__[arg].default
                        self.model_args[arg] = default
                        self.param.add_parameter(arg, param.Parameter(default, per_instance=True))
                    else:
                        self.param.add_parameter(arg, param.Parameter(val, per_instance=True))
                    # self.model_args_list.append(arg)
                else:
                    raise ValueError(f"'{arg}' is missing from the model class's signature")
                self.param.watch(self._create_model, [*self.model_args.keys()])


    def _create_model(self, event=None):
        """Creates the model instance on init and when any of the parameters change"""
        arg_vals = {arg: self.param.values()[arg] for arg in self.model_args.keys()}
        self.model = self.model_class(**arg_vals)

In [373]:
c = ChatModel(model_class=ChatOpenAI, model_args={'model_name': 'gpt4'})
c

ChatModel(model=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f9371244290>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f93707c72f0>, model_name='gpt4', openai_api_key=SecretStr('**********'), openai_proxy=''), model_args={'model_name': 'gpt4'}, model_class=<class 'langchain_openai.chat_models.base.ChatOpenAI'>, model_name='gpt4', name='ChatModel00804')

## Corpus

In [ ]:
class Corpus(param.Parameterized):
    name = param.String()
    corpus_creation_args = param.Dict(default={})
    corpus_retrieval_args = param.Dict(default={})
    
    # Creation logic
    # Retrieval logic
    # Storage logic - is it persistent? option?
    

## RAG

In [230]:
class ParameterizedABCMeta(ABCMeta, type(Parameterized)):
    pass

class BaseParameterized(Parameterized, metaclass=ParameterizedABCMeta):
    pass

class BaseRAG(BaseParameterized):
    # add params as necessary for external use
    @abstractmethod
    def create_corpus(self):
        pass

class SimpleRAG(BaseRAG):

    def create_corpus(self):
        pass

In [236]:
class BaseRAG(ABC):
    # add params as necessary for external use
    @abstractmethod
    def create_corpus(self):
        pass
class NewRAG(param.Parameterized, BaseRAG):

    def __init__(self):
        print('initd')

new_rag = NewRAG()

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [256]:
class ParameterizedABCMeta(ABCMeta, type(param.Parameterized)):
    pass

class BaseRAG(param.Parameterized, metaclass=ParameterizedABCMeta):
    # add params as necessary for external use
    @abstractmethod
    def create_corpus(self):
        pass
    
    def tangible_method(self):
        print('something')


class NewRAG(BaseRAG):
    some_param = param.String('stringy')

    def __init__(self):
        print('initd')

    def create_corpus(self):
        pass

new_rag = NewRAG()

initd


In [249]:
new_rag.param

In [235]:
param.__version__

'2.1.0'

## Memory

# Skeleton Pipeline

## Load and split

In [2]:
def token_len(text, model_name='gpt-3.5-turbo'):
    encoding = tiktoken.encoding_for_model(model_name)
    encoding_len = len(encoding.encode(text))
    return encoding_len

In [3]:
tpdf_path = Path('examples/data/2112.01488v3.pdf')
tpdf_loader = UnstructuredPDFLoader(tpdf_path)
tpdf_doc = tpdf_loader.load()

tpdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50, length_function=token_len)
tpdf_chunks = tpdf_splitter.split_documents(tpdf_doc)
tpdf_chunks

[Document(page_content='2 2 0 2\n\nl u J\n\n0 1\n\n]\n\nR\n\nI . s c [\n\n3 v 8 8 4 1 0 . 2 1 1 2 : v i X r a\n\nColBERTv2: Effective and Efﬁcient Retrieval via Lightweight Late Interaction\n\nKeshav Santhanam∗ Stanford University\n\nOmar Khattab∗ Stanford University\n\nJon Saad-Falcon Georgia Institute of Technology\n\nChristopher Potts Stanford University\n\nMatei Zaharia Stanford University\n\nAbstract\n\nNeural information retrieval (IR) has greatly advanced knowledge- and intensive language tasks. While many neural IR methods encode queries and documents into late interaction models produce multi-vector repre- sentations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inﬂates the space footprint of these models by an order of magnitude. In this work, we introduce ColBERTv2, a retriever that couples an aggressive residual compres- sion mechanis

## Encoding

In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# For a retrieval task that uses short queries to find long related documents,
# it is recommended to add instructions for these short queries. 
emb_model_name = 'BAAI/bge-large-en-v1.5' 
emb_model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=emb_model_name, model_kwargs=emb_model_kwargs, encode_kwargs=encode_kwargs
)
hf


/root/miniconda3/envs/RAGular/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-large-en-v1.5', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='')

In [5]:
embedded_chunks = hf.embed_documents([_.page_content for _ in tpdf_chunks]) # Takes a list of strings instead of documents(???)
embedded_chunks

[[0.027144655585289,
  0.010908291675150394,
  -0.007803196553140879,
  0.010273820720613003,
  -0.0004490670107770711,
  -0.005330324172973633,
  -0.03391449153423309,
  -0.04463495314121246,
  -0.01265870314091444,
  0.04743312671780586,
  0.0319766141474247,
  -0.012839773669838905,
  -0.03520369529724121,
  -0.04842870309948921,
  -0.017946913838386536,
  -0.05331189185380936,
  -0.011299281381070614,
  -0.025722434744238853,
  -0.04562721028923988,
  -0.03869546577334404,
  0.028745662420988083,
  0.01883101277053356,
  -0.06315965950489044,
  0.010715173557400703,
  -0.0074507747776806355,
  0.010830319486558437,
  0.02072223089635372,
  0.02249465137720108,
  0.08006232976913452,
  0.04530643671751022,
  0.0470992811024189,
  -0.05207795649766922,
  -0.0023492886684834957,
  -0.017810437828302383,
  -0.00837847962975502,
  -0.06615505367517471,
  0.03105706349015236,
  -0.005657607689499855,
  0.012417525053024292,
  -0.036523349583148956,
  0.0315505750477314,
  -0.033478498458

## Vector Store


In [430]:
# Corpus folder creation
corpus_local = Path('corpus_local')
simple_rag_path = corpus_local / 'simple_rag'
simple_rag_path.mkdir(parents=True, exist_ok=True)

In [ ]:
chroma = Chroma(collection_name='simple_rag0', embedding_function)



In [7]:
# TODO: create new client each time or have a pool of clients?
client = chromadb.PersistentClient(path=str(simple_rag_path / 'chromadb'))

In [62]:
collection = client.get_or_create_collection()


In [67]:
hf.embed_documents

<bound method HuggingFaceBgeEmbeddings.embed_documents of HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-large-en-v1.5', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='')>

In [8]:
collection = client.get_or_create_collection(name='simple_rag')

In [80]:
tpdf_chunks

[Document(page_content='2 2 0 2\n\nl u J\n\n0 1\n\n]\n\nR\n\nI . s c [\n\n3 v 8 8 4 1 0 . 2 1 1 2 : v i X r a\n\nColBERTv2: Effective and Efﬁcient Retrieval via Lightweight Late Interaction\n\nKeshav Santhanam∗ Stanford University\n\nOmar Khattab∗ Stanford University\n\nJon Saad-Falcon Georgia Institute of Technology\n\nChristopher Potts Stanford University\n\nMatei Zaharia Stanford University\n\nAbstract\n\nNeural information retrieval (IR) has greatly advanced knowledge- and intensive language tasks. While many neural IR methods encode queries and documents into late interaction models produce multi-vector repre- sentations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inﬂates the space footprint of these models by an order of magnitude. In this work, we introduce ColBERTv2, a retriever that couples an aggressive residual compres- sion mechanis

In [12]:
def kv_to_str(input_dict):
    return {str(k): str(v) for k, v in input_dict.items()}

metadata = kv_to_str(tpdf_chunks[0].metadata)
metadata


{'source': 'examples/data/2112.01488v3.pdf'}

In [14]:
collection.add(ids=[str(uuid4()) for _ in embedded_chunks],
               embeddings=embedded_chunks,
               documents=[_.page_content for _ in tpdf_chunks],
               metadatas=[kv_to_str(_.metadata) for _ in tpdf_chunks])

In [70]:
client.delete_collection('simple_rag')

In [23]:
type(client)

chromadb.api.client.Client

In [49]:
tpdf_chunks[0]

Document(page_content='2 2 0 2\n\nl u J\n\n0 1\n\n]\n\nR\n\nI . s c [\n\n3 v 8 8 4 1 0 . 2 1 1 2 : v i X r a\n\nColBERTv2: Effective and Efﬁcient Retrieval via Lightweight Late Interaction\n\nKeshav Santhanam∗ Stanford University\n\nOmar Khattab∗ Stanford University\n\nJon Saad-Falcon Georgia Institute of Technology\n\nChristopher Potts Stanford University\n\nMatei Zaharia Stanford University\n\nAbstract\n\nNeural information retrieval (IR) has greatly advanced knowledge- and intensive language tasks. While many neural IR methods encode queries and documents into late interaction models produce multi-vector repre- sentations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inﬂates the space footprint of these models by an order of magnitude. In this work, we introduce ColBERTv2, a retriever that couples an aggressive residual compres- sion mechanism

In [28]:
results = collection.query(query_embeddings=hf.embed_documents(['What is the capital of France?']))
results


{'ids': [['720959dc-4f12-4ad5-841b-f54403baf9d4',
   'dca00914-63f4-44c7-9ada-7376f20dcdf0',
   'a29c4a1b-c07d-4849-b78b-3cca11edd539',
   '1f91b043-3937-4d78-b74e-c01486a0366a',
   '5b37a974-b6db-467f-81d5-0614bf259a8c',
   '1f1ed0de-ccc1-450c-86d2-74cf04664e70',
   '08f09ebd-8910-4092-bfee-f065e80f3c2a',
   '68764425-eb7b-4779-9548-c0e819f7b592',
   'efcb04b2-2094-42f1-b3f3-79b1b7ae8a36',
   '28b8af28-55ca-44dd-bbd1-bdc9057bfb5f']],
 'distances': [[1.2179785925348834,
   1.2820421072387942,
   1.283722624762129,
   1.3394140314613365,
   1.3476100072530883,
   1.3497488412458007,
   1.3501860928900984,
   1.3516339373651218,
   1.3527374372908103,
   1.3601365266493244]],
 'metadatas': [[{'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'sourc

In [38]:
results['metadatas'][0]

[{'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'}]

In [29]:
results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

## Retriever

In [51]:
from typing import Optional, Callable, List
from langchain_core.documents import Document
from chromadb.api.client import Client
from chromadb.api.models.Collection import Collection

class SimpleRAGRetriever(BaseRetriever):
    vector_store_client: Client = None
    collection_name: str = None
    # doc_embedding_function = None # probably don't need
    query_embedding_function: Optional[Callable[[str], List[float]]] = None
    collection: Collection = None
    k: int = 10
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.collection = self.vector_store_client.get_or_create_collection(name=self.collection_name)

    def _get_relevant_documents(self, query, run_manager):
        embedded_query = self.query_embedding_function(query)
        results = self.collection.query(query_embeddings=embedded_query, n_results=self.k)
        docs = []
        for n in range(len(results['ids'])):
            doc = Document(
                page_content=results['documents'][0][n],
                metadata=results['metadatas'][0][n]
                )
            docs.append(doc)
        return docs

In [52]:
simple_rag_retriever = SimpleRAGRetriever(vector_store_client=client, collection_name='simple_rag', query_embedding_function=hf.embed_query)
# 


In [53]:
# simple_rag_retriever.invoke('What is the capital of France?')



[Document(page_content='Subtopics\n\nWriting\n\nSearch Forum\n\n497 2003\n\n277k\n\nESL, Linguistics, Worldbuilding\n\n1071 2000\n\n200k\n\nEnglish\n\nRecreation\n\nSearch Forum\n\n563 2002\n\n263k\n\nSci-Fi, RPGs, Photography\n\n924 2002\n\n167k\n\nGaming, Anime, Movies\n\nScience\n\nSearch Forum\n\n538 2013\n\n344k\n\nChemistry, Statistics, Academia\n\n617 2017\n\n1.694M\n\nMath, Physics, Biology\n\nTechnology\n\nSearch Forum\n\n916 2003\n\n1.276M\n\nWeb Apps, Ubuntu, SysAdmin\n\n596 2004\n\n639k\n\nApple, Android, UNIX, Security\n\nLifestyle\n\nSearch Forum\n\n417 2076\n\n269k\n\nDIY, Music, Bicycles, Car Maintenance\n\n661 2002\n\n119k\n\nCooking, Sports, Travel\n\nPooled\n\nSearch Forum\n\n2931 10097\n\n2.4M\n\nAll of the above\n\n3869 10025\n\n2.8M All of the above\n\nTable 1: Composition of LoTTE showing topics, question sets, and a sample of corresponding subtopics. Search Queries are taken from GooAQ, while Forum Queries are taken directly from the StackExchange archive. The p

## Chat pipe

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory


In [2]:
from langchain_core.messages import HumanMessage
HumanMessage('wassup').dict()

{'content': 'wassup',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': None,
 'example': False}

In [428]:
ChatOpenAI(model='gpt-3.5-turbo')

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f936f640680>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f936f6431d0>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [447]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a funny assistant"),
        ("human", "{yapping}")
    ]
)

chain = prompt | ChatOpenAI(model='gpt-3.5-turbo')

rtrn_msg = chain.invoke({'yapping': 'what is the capital of France?'})
rtrn_msg

AIMessage(content='The capital of France is "F." Just kidding! It\'s Paris.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 23, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-10024577-732c-45ab-9e4d-0efad8b050e3-0')

In [498]:
class ConversationAgent(param.Parameterized):
    history_messages = param.List(per_instance=True)
    system_prompt = param.String(default='You are a funny assistant', per_instance=True)
    
    def send(self, message):
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_prompt),
                MessagesPlaceholder('history'),
                ("human", "{message}")
            ]
        )
        chain = prompt | ChatOpenAI(model='gpt-3.5-turbo')
        reply = chain.invoke({'history': self.history_messages, 'message': message})
        self.history_messages.extend([('human', message), ('assistant', reply.content)])
        return reply



# Framework Construction

## Ports

In [102]:
# %%writefile /workspaces/pyllments/pyllments/ports/ports.py
from typing import TYPE_CHECKING, Union, List

# import param

# from pyllments.base import Payload



class Port(param.Parameterized):
    payload = param.ClassSelector(class_=Payload)
    containing_element = param.Parameter(default=None)
    connected_elements = param.List()

class InputPort(Port):
    subject_ports = param.List(item_type=Port)
    unpack_payload_callback = param.Callable(default=None)

    def receive(self, payload):
        pass

    def unpack_payload(self, payload):
        pass

class OutputPort(Port):
    """
    Handles the intake of data and packing into a payload and
    is meant to connect to an InputPort in order to emit the packed payload
    """
    
    required_objects = param.List(item_type=param.Parameterized)
    emit_when_ready = param.Boolean(default=True)

    observer_ports = param.List(item_type=Port)
    pack_payload_callback = param.Callable(default=None)
    staged_objects = param.List()
    
    def pack_payload(self):
        if self.pack_payload_callback:
            return self.pack_payload_callback()
        else:
            return self.payload_type()

    def emit(self):
        """Packs the payload and emits it to all registered observers"""
        for port in self.observer_ports:
            port.receive(payload)
    
    def stage(self, params: Union[List[Parameter], Parameter]):
        pass
    
    def stage_and_emit(self, params):
        """Stages the payload and emits it - All required params need be present"""
        self.stage()
        self.emit()

    def connect(self, other):
        """Connects self and the InputPort"""
        if not isinstance(other, InputPort):
            raise ValueError('Can only connect OutputPorts to InputPorts')
        if not isinstance(other.payload, type(self.payload)):
            raise ValueError('InputPort and OutputPort payload types must match')
        
        self.observer_ports.append(other)
        other.connected_elements.append(self.containing_element)

    def __gt__(self, other):
        """Implements self.connect(other) through el1.some_input > el2.some_output"""
        self.connect(self, other)
        

class Ports(param.Parameterized):
    """Keeps track of InputPorts and OutputPorts and handles their creation"""
    input = param.List(item_type=InputPort)
    output = param.List(item_type=OutputPort)
    containing_element = param.Parameter(default=None)

    def add_input(self, name, payload_type, ):
        input_port = InputPort(
            name=name,
            payload=payload_type(),
            containing_element=self.containing_element
            )
        self.param.add_parameter(name, param.Parameter(input_port))
        self.input.append(input_port)
        return input_port
    
    def add_output(self, name, payload_type, ):
        output_port = OutputPort(
            name=name,
            payload=payload_type(),
            containing_element=self.containing_element
            )
        self.output.append(output_port)
        return output_port


In [96]:
# p = OutputPort(payload=MessagePayload)
ps = Ports()

ps.add_input('test_payload', MessagePayload)


InputPort(connected_elements=[], containing_element=None, name='test_payload', payload=MessagePayload(message_type=None, message_view=None, model=None, name='MessagePayload00185'), subject_ports=[])

In [106]:
class Test(param.Parameterized):
    a = param.Number()

test = Test(a=3.14)
OutputPort(
    payload=MessagePayload(),
    required_objects=[test.a]
    )



OutputPort(connected_elements=[], containing_element=None, emit_when_ready=True, name='OutputPort00194', observer_ports=[], pack_payload_callback=None, payload=MessagePayload(message_type=None, message_view=None, model=None, name='MessagePayload00193'), required_objects=[3.14], staged_objects=[])

In [104]:
import param

class U(param.Parameterized):
    
    a = param.Number()
    
class V(param.Parameterized):
    
    b = param.Number(default=None, allow_refs=True)

u = U(a=3.14)
v = V()
v.b = u.param.a

v.b

3.14

In [97]:
ps.test_payload.payload

MessagePayload(message_type=None, message_view=None, model=None, name='MessagePayload00185')

## Base

In [39]:
# %%writefile /workspaces/pyllments/pyllments/base/element.py
# import param

# from pyllments.ports import InputPorts, OutputPorts

class Element(param.Parameterized):
    model = param.ClassSelector(class_=Model, default=None)
    ports = param.ClassSelector(class_=Ports, default=Ports())

    def __init__(self, **params):
        super().__init__(**params)
        self.ports.containing_element = self

In [40]:
el = Element()
el.ports.containing_element

Element(name='Element00150', ports=Ports(containing_element=..., input=[], name='Ports00151', output=[]))

In [30]:
e1 = element1.ports.add_input('input_name', Payload)
e2 = element2.ports.add_output('output_name', Payload)

e1.ports.output_name > e2.ports.input_name

In [60]:
# %%writefile /workspaces/pyllments/pyllments/base/model.py
# import param

class Model(param.Parameterized):
    def __init__(self, **params):
        super().__init__(**params)

In [58]:
# %%writefile /workspaces/pyllments/pyllments/base/payload.py
# import param

class Payload(param.Parameterized):
    pass

In [47]:
# %%writefile /workspaces/pyllments/pyllments/base/listing.py
# import param

class Listing(param.Parameterized):
    def __init__(self, **params):
        super().__init__(**params)

Writing /workspaces/pyllments/pyllments/base/listing.py


## Payload

### Message Payload

In [62]:
# %%writefile /workspaces/pyllments/pyllments/payloads/message_payload/message_model.py
# import param

# from pyllments.base import Model

class MessageModel(Model):
    message = param.String(default=None, per_instance=True)
    message_type = param.String(default=None, per_instance=True)

    # payload_pack

In [63]:
# %%writefile /workspaces/pyllments/pyllments/payloads/message/message_payload.py
# import param
# import panel as pn

# from pyllments.base import Payload
# from pyllments.payloads.message.message_model import MessageModel

class MessagePayload(Payload):
    model = param.ClassSelector(class_=MessageModel, is_instance=True)
    message_type = param.String(default=None, per_instance=True)
    message_view = param.ClassSelector(class_=pn.pane.Markdown, is_instance=True)

    def create_message_view(self, **kwargs):
        self.message_view = pn.pane.Markdown(self.message)
        return self.message_view

## ChatInterface

### ChatInterfaceModel

In [6]:
%%writefile /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_model.py
import param

from pyllments.base import Model
from pyllments.payloads.message import MessagePayload

class ChatInterfaceModel(Model):
    message_list = param.List(default=[], per_instance=True)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.param.add_parameter(
            'created_message',
            param.ClassSelector(class_=MessagePayload)
            )
        self.message_list.append(('human', 'what is the capital of the moon?'))

Overwriting /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_model.py


### ChatInterfaceElement

In [2]:
# %%writefile /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_element.py
# import panel as pn
# import param

# from pyllments.base import Element, Model
# from pyllments.elements.chat_interface import ChatInterfaceModel

class ChatInterfaceElement(Element):
    '''Responsible for creating the chat feed, input, and send button views'''
    model = param.ClassSelector(class_=Model, default=ChatInterfaceModel())

    chatfeed_view = param.ClassSelector(class_=pn.chat.ChatFeed, is_instance=True)
    chat_input_view = param.ClassSelector(class_=pn.chat.ChatAreaInput, is_instance=True)
    send_button_view = param.ClassSelector(class_=pn.widgets.Button, is_instance=True)

    def create_chatfeed_view(self, **kwargs):
        self.chatfeed_view = pn.chat.ChatFeed(**kwargs)
        return self.chatfeed_view
    
    def create_chat_input_view(self, **kwargs):
        self.chat_input_view = pn.chat.ChatAreaInput(**kwargs)
        return self.chat_input_view

    def create_send_button_view(self, **kwargs):
        self.send_button_view = pn.widgets.Button(**kwargs)
        return self.send_button_view

    def _on_send(self, event):
        self.chat_interface_model.message_list.append(('human', self.chat_input_view.value))
        self.chat_input_view.value = ''
        self.chatfeed_view.update()

Overwriting /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_element.py


In [3]:
from pyllments.elements.chat_interface import ChatInterfaceElement

## LLM Chat

### LLM Chat Model

In [4]:
# %%writefile /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_model.py
# import param

# from langchain_core.language_models import BaseLanguageModel
# from langchain_openai import ChatOpenAI

# from pyllments.base import Model

class LLMChatModel(Model):
    model_class = param.ClassSelector(
        class_=BaseLanguageModel,
        is_instance=False,
        doc='Class of a LangChain Chat Model',
        default=ChatOpenAI
    )
    model_args = param.ClassSelector(
        class_=(dict, list), #TODO: May replace with dict, as the elements will the logic for view creation
        doc='''Takes a dictionary of arguments to pass to expose and send to the
        model class. If you set a None value as the key, the argument will be exposed,
        with the default value set. Passing a list is the same as passing a dict
        with all values set to None.''',
        is_instance=True,
        default={}
        )

    model = param.ClassSelector(class_=BaseLanguageModel, default=None, pickle_default_value=False)

    def __init__(self, **params):
        super().__init__(**params)

        self._set_params()
        self._create_model()

    def _set_params(self):
        """Sets specified model_args as params of the object"""
        if self.model_args:
            for arg, val in self.model_args.items():
                if arg in self.model_class.__fields__:
                    if val is None:
                        default = self.model_class.__fields__[arg].default
                        self.model_args[arg] = default
                        self.param.add_parameter(arg, param.Parameter(default, per_instance=True))
                    else:
                        self.param.add_parameter(arg, param.Parameter(val, per_instance=True))
                    # self.model_args_list.append(arg)
                else:
                    raise ValueError(f"'{arg}' is missing from the model class's signature")
                self.param.watch(self._create_model, [*self.model_args.keys()])


    def _create_model(self, event=None):
        """Creates the model instance on init and when any of the parameters change"""
        arg_vals = {arg: self.param.values()[arg] for arg in self.model_args.keys()}
        self.model = self.model_class(**arg_vals)

    def stream(self, messages):
        # TODO: Set it up to be async
        return self.model.stream(messages)

Writing /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_model.py


In [1]:
from pyllments.elements.llm_chat import LLMChatModel

### LLM Chat Element

In [6]:
# %%writefile /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_element.py
# import param
# import panel as pn

# from pyllments.base import Element
# from pyllments.elements.llm_chat import LLMChatModel

class LLMChatElement(Element):
    model = param.ClassSelector(class_=LLMChatModel)

    temperature = param.Number(default=0.5)
    temperature_view = param.ClassSelector(class_=pn.widgets.FloatSlider, is_instance=True)

    def __init__(self, **params):
        super().__init__(**params)

    def create_temperature_view(self, **kwargs):
        self.temperature_view = pn.widgets.FloatSlider(**kwargs)
        return self.temperature_view

Writing /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_element.py


In [9]:
from pyllments.elements.llm_chat import LLMChatElement

### LLM Chat Listing

In [10]:
%%writefile /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_listing.py
import param
import panel as pn
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

from pyllments.base import Listing
from pyllments.elements.llm_chat import LLMChatElement, LLMChatModel

class LLMChatListing(Listing):
    selector = param.ClassSelector(class_=pn.widgets.Select, is_instance=True)
    llm_chat_element = param.ClassSelector(class_=LLMChatElement, is_instance=True)
    chat_model_dict = param.Dict(default={
        'gpt-3.5-turbo': ChatOpenAI,
        'gpt-4o': ChatOpenAI,
        'claude-3-opus-20240229': ChatAnthropic,
    })
    model_args = param.Dict(default={
        'temperature': 0.5
    })
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        default_model_name = next(iter(self.chat_model_dict))
        self.create_llm_chat_element(default_model_name)

    def create_llm_chat_element(self, chat_model_name):
        """Creates and sets the chat model element"""
        llm_chat_model = LLMChatModel(
            model_class=self.chat_model_dict[chat_model_name],
            model_args=self.model_args | {'model_name': chat_model_name}
            )
        self.llm_chat_element = LLMChatElement(llm_chat_model=llm_chat_model)
        return self.llm_chat_element

    def create_selector(self):
        self.selector = pn.widgets.Select(options=list(self.chat_model_dict.keys()))
        self.selector.param.watch(self._on_selector_change, 'value')
        return self.selector
    
    def _on_selector_change(self, event):
        """Callback method triggered when the selector value changes"""
        # print(event)
        self.create_llm_chat_element(event.new)


Writing /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_listing.py


In [1]:
from pyllments.elements.llm_chat import LLMChatListing

In [245]:
chat_listing = LLMChatListing()

chat_listing.create_selector()
chat_listing.selector

BokehModel(combine_events=True, render_bundle={'docs_json': {'2d711ee2-5b70-4054-adda-f61d0e46ab54': {'version…

### Chat

In [3]:
class ConversationHistory:
    pass

class ContextHandler:
    pass

class RetrieverStore:
    pass

class ReplyHandler:
    pass


In [221]:
ChatInterfaceElement.param

In [55]:
# Model layer
class Chat(param.Parameterized):
    conversation_history = param.ClassSelector(
       class_=ConversationHistory,
       default=None,
       doc="Message handler between AI and User - could contain source messages as well")
    context_history = param.ClassSelector(
        class_=ContextHandler,
        default=None,
        is_instance=True,
        doc="Context handler")
    use_context = param.Boolean(
        default=False,
        doc="Whether to use context from the context handler"
    )
    retriever_store = param.ClassSelector(
        class_=RetrieverStore,
        default=None,
        is_instance=True,
        doc="Retriever store")
    retrieval = param.ClassSelector(
        class_=BaseRAG,
        default=None,
        is_instance=True,
        doc="RAG handler")
    reply = param.ClassSelector(
        class_=ReplyHandler,
        default=None,
        is_instance=True,
        doc="Last Generated Reply")
    # sources = param.List(default=[], doc="List of sources to retrieve")
    prompt_template = param.ClassSelector(
        class_=ChatPromptTemplate,
        is_instance=True,
        doc="Prompt template for the chat model")
    system_message = param.String(
        default="You're a helpful assistant",
        doc="System message for the prompt"
    )
    chat_model = param.ClassSelector(
        class_=LLMChatModel,
        is_instance=True,
        doc="Chat model"
    )

    
    def generate_reply(self, message, astream=True):
        """"""
        # sources = self.rag.get_sources(message) 
        # context = self.context_history.get_context(sources)
        messages = [SystemMessage(self.system_message), HumanMessage(message)]
        if self.use_context:
            messages.insert(1, 'history goes here')
        prompt_template = ChatPromptTemplate.from_messages(messages)
        chain = prompt_template | self.chat_model.model
        if astream:
            stream = chain.astream({'message': message})
            return stream
        else:
            return chain.stream({'message': message})

In [56]:
openai_chat_model = ChatOpenAI
llm_chat_model = LLMChatModel(model_class=ChatOpenAI, model_args={'model_name': 'gpt-3.5-turbo'})

In [57]:
chat = Chat(
    chat_model=llm_chat_model,
    system_message='You are a posh model that answers questions verbosely.')

In [173]:
reply_stream = chat.generate_reply('what is the capital of the moon?', astream=True)
md = pn.pane.Markdown('# ')
cm = pn.chat.ChatMessage(md, user="Wise guy", avatar="🤓")
cm

BokehModel(combine_events=True, render_bundle={'docs_json': {'7b91a5e7-4a10-42fc-83e8-9b1a58f2582e': {'version…

In [175]:
async for token in reply_stream:
    md.object += f' {token.content}'

In [90]:
type(reply_stream)

async_generator

In [170]:
md = pn.pane.Markdown('# HI')
cm = pn.chat.ChatMessage(md, user="Wise guy", avatar="🤓")
cm

BokehModel(combine_events=True, render_bundle={'docs_json': {'8c45e048-8a01-4e98-8bbf-f27c2bda892b': {'version…

In [10]:
# View Layer
class ChatView(Viewer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.chat_input = pn.chat.ChatAreaInput()
        self.chat_feed = pn.chat.ChatFeed()
        self.chat_send_button = pn.widgets.Button(name='Send')
        self.chat_retrieve_switch = pn.widgets.Switch(name='Retrieve')
        self.chat_list = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.corpus_selector = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        # self.CorpusParametersView = CorpusParametersView()
        
    
    def __panel__(self):
        return pn.Column(self.chat_feed, self.chat_input)

In [ ]:
class ChatController(param.Parameterized):
    chat_view = param.

In [ ]:
class CorpusManager(param.Parameterized):
    corpus_list = param.List(default=[])
    selected_corpus = param.String()
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        corpus_list = self.get_corpora()
    
    def get_corpora(self):
        corpora_path = BASE_DIR / 'corpora'
        return [f.name for f in corpora_path.iterdir() if f.is_dir()]
        



## View Layer

In [11]:
pn.panel(ChatView())

BokehModel(combine_events=True, render_bundle={'docs_json': {'9d07223d-8e03-49c3-a46f-cdfb05e194f2': {'version…

In [ ]:
class RetrieverCreationView(Viewer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.retriever_type = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.corpus_selector = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.CorpusParametersView = CorpusParametersView()
        
    def __panel__(self):
        return pn.Column(self.retriever_type, self.corpus_selector, self.CorpusParametersView)



## Controller Layer

In [ ]:
# Controller Layer
class ChatController(param.Parameterized):
    view = param.ClassSelector(class_=ChatView, is_instance=True)
    model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)

    def __panel__(self):
        return self.view()

In [26]:
pn.panel(ChatInterface())

BokehModel(combine_events=True, render_bundle={'docs_json': {'c0cc8570-7eb7-4dcb-9412-cd85afe5b670': {'version…

In [45]:
def on_click(event):
    print("ChatMessage clicked!")

message = pn.chat.ChatMessage(pn.Row(pn.widgets.Button(name='click me', height=50), pn.pane.Markdown('# some test'), pn.widgets.Button(name='click me', height=50),  pn.pane.Markdown('# some test')))
serve(message)

Launching server at http://localhost:35594


In [50]:
chat_feed.send('interior crocadile alligator')

BokehModel(combine_events=True, render_bundle={'docs_json': {'682c0bce-e79a-4b7f-9753-e8290db425be': {'version…

In [48]:
import asyncio

In [65]:
chat_feed = pn.chat.ChatFeed()

# creates a new message
message = chat_feed.stream("Hello", user="Aspiring User", avatar="🤓")
chat_feed

BokehModel(combine_events=True, render_bundle={'docs_json': {'e465d4fe-140f-4671-8dac-d50f2ef73d36': {'version…

In [67]:
# streams (appends) to the previous message
message = chat_feed.stream(
    'what is the capital of France?',
    user="reeeee",
    avatar="🤓",
    message=message,
)

In [64]:
# streams (appends) to the previous message
message = chat_feed.stream(
    pn.pane.Markdown('is it me'),
    user="reeeee",
    avatar="🤓",
    message=message,
)

# Project initialization scripts

In [ ]:
# cli.py

import argparse

def create_project():
    """Establishes a new project directory structure and config"""
    parser = argparse.ArgumentParser(description='Initialize a new project')
    parser.add_argument('name', type=str, help='Name of the project')
    parser.add_argument('--include-examples', action='store_true', help='Include examples in the project')
    args = parser.parse_args()

    project_path = Path.cwd() / args.name
    project_path.mkdir(parents=True, exist_ok=True)
    
    (project_path / "corpora").mkdir(exist_ok=True)
    (project_path / "retrievers").mkdir(exist_ok=True)
    (project_path / "memory").mkdir(exist_ok=True)
    (project_path / "files").mkdir(exist_ok=True)
    (project_path / "retrieval_database").mkdir(exist_ok=True)

    with open(project_path / "project.yaml", 'w') as file:
        yaml = YAML()
        data = {'include_examples': args.include_examples}
        yaml.dump(data, file)



# App Creation

In [ ]:
def app_run():
    

    pn.serve(app)

